In [ ]:
print("Installation/Importation des librairies necessaires.")

In [ ]:
%%capture capt
!pip install jellyfish
!pip install levenshtein

In [ ]:
%%capture capt
import numpy as np
from jellyfish import jaro_winkler_similarity
import pandas as pd
from tqdm import tqdm
import Levenshtein
import math

# Formules de distance entre string

##### Jaro-Winkler

In [ ]:
def convert_to_sentence(arr):
    sentence = ''
    for x in arr:
        sentence += x + ","
    return sentence[:-1]

In [ ]:
def string_distance(s1, s2):
    # Compute distance between strings using the jaro winkler similarity formula
    return 1 - jaro_winkler_similarity(s1, s2)

def jaro_winkler_distance(arr1, arr2):
    s1 = convert_to_sentence(arr1)
    s2 = convert_to_sentence(arr2)
    return abs(string_distance(s1, s2))

##### Levenshtein 

In [ ]:
def string_distance_levenshtein(s1, s2):
    return 1 - Levenshtein.ratio(s1, s2)

def levenshtein_distance(arr1, arr2):
    s1 = convert_to_sentence(arr1)
    s2 = convert_to_sentence(arr2)
    return abs(string_distance_levenshtein(s1, s2))

##### Custom

Tous les GHM ont 6 caractères

In [ ]:
def custom_ghm_distance_norm(ghm1, ghm2, distance=string_distance_levenshtein, weights=[50,10,1]):
    return (weights[0]*distance(ghm1[:2], ghm2[:2]) + weights[1]*distance(ghm1[2], ghm2[2]) + weights[2]*distance(ghm1[2:], ghm2[2:])) / np.sum(weights)

def custom_distance_norm(arr1, arr2, distance_ghm=string_distance_levenshtein, weights=[50, 10, 1]):
    min_length = min(len(arr1), len(arr2))
    max_length = max(len(arr1), len(arr2))
    distance = np.sum([custom_ghm_distance_norm(arr1[i], arr2[i], distance=distance_ghm,weights=weights) for i in range(min_length)]) + (max_length - min_length)
    return distance / max_length

In [ ]:
def custom_ghm_distance(ghm1, ghm2, distance=string_distance_levenshtein, weights=[50,10,1]):
    return (weights[0]*distance(ghm1[:2], ghm2[:2]) + weights[1]*distance(ghm1[2], ghm2[2]) + weights[2]*distance(ghm1[2:], ghm2[2:]))

def custom_distance(arr1, arr2, distance_ghm=string_distance_levenshtein,weights=[50,10,1]):
    min_length = min(len(arr1), len(arr2))
    max_length = max(len(arr1), len(arr2))
    distance = np.sum([custom_ghm_distance(arr1[i], arr2[i], distance=distance_ghm,weights=weights) for i in range(min_length)]) + (max_length - min_length)
    return distance